In [ ]:
import importlib
import datetime
import random
import math

import folium
import tqdm
import haversine
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN, KMeans
from nltk.cluster.kmeans import KMeansClusterer
from sklearn.metrics import pairwise_distances

from projects.common import geo
from projects.common.nile import dates, test_utils
from projects.cpo_model import project_config, default_values, common

In [ ]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

In [ ]:
_ = importlib.reload(default_values)
_ = importlib.reload(project_config)
_ = importlib.reload(common)
_ = importlib.reload(test_utils)
_ = importlib.reload(geo)

In [ ]:
cluster = project_config.get_project_cluster(
    parallel_operations_limit=5,
    pool='taxi-delivery',
    extra_requirements=[],
)

### Define functions

In [ ]:
def convert_to_datetime(dttm_str):
    return datetime.datetime.strptime(dttm_str, '%Y-%m-%d %H:%M:%S')

def choose_random_time_in_interval(dttm_left, dttm_right):
    delta = (dttm_right - dttm_left).total_seconds()
    return dttm_left + datetime.timedelta(seconds=random.randint(0, delta))

def parse_gps_str(gps_str):
    return tuple(float(c) for c in gps_str.split(','))

def assign_day(dttm, month, day):
    return dttm.replace(month=month, day=day)

### Get and prepare dataframes

In [ ]:
# market_orders_df = test_utils.decode_dataframe(
#     cluster
#     .read('//home/taxi-delivery/analytics/dev/cpo_model/moscow_deliveries_2020-10-29_2020-11-04')
#     .as_dataframe()
# )
# market_orders_df = market_orders_df[~pd.isnull(market_orders_df['final_delivery_point_gps'])]

market_orders_df = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/cpo_model/kazan_deliveries_2020')
    .as_dataframe()
)
market_orders_df = market_orders_df[~pd.isnull(market_orders_df['final_delivery_point_gps'])]

In [ ]:
market_orders_df.head(2)

In [ ]:
# lavka_places_df = test_utils.decode_dataframe(
#     cluster
#     .read('//home/taxi-delivery/analytics/dev/cpo_model/lavka_places_moscow')
#     .as_dataframe()
# )
# lavka_places_df = lavka_places_df[pd.isnull(lavka_places_df['disable_details_status'])].reset_index()

In [ ]:
market_orders_df['original_delivery_from_datetime'] = (
    market_orders_df['original_delivery_from_datetime'].apply(
        lambda x: x[:10] + ' ' + '09:00:00' if x[11:] < '09:00:00' else x
    ).apply(
        lambda x: assign_day(convert_to_datetime(x), 11, 2)
    )
)
market_orders_df['original_delivery_to_datetime'] = (
    market_orders_df['original_delivery_to_datetime'].apply(
        lambda x: x[:10] + ' ' + '23:00:00' if x[11:] > '23:00:00' else x
    ).apply(
        lambda x: assign_day(convert_to_datetime(x), 11, 2)
    )
)

In [ ]:
market_orders_df = market_orders_df[market_orders_df['delivery_type']=='Курьерская доставка']

In [ ]:
# Market Orders.
market_deliveries_df = pd.DataFrame()
market_deliveries_df['service_name'] = ['market']*len(market_orders_df)
market_deliveries_df['weight'] = market_orders_df['order_physical_weight'].fillna(
    np.mean(market_orders_df['order_physical_weight'])
)
market_deliveries_df['volume'] = None
market_deliveries_df['delivery_lat'] = (
    market_orders_df['final_delivery_point_gps'].apply(lambda x: parse_gps_str(x)[1])
)
market_deliveries_df['delivery_lon'] = (
    market_orders_df['final_delivery_point_gps'].apply(lambda x: parse_gps_str(x)[0])
)
market_deliveries_df['warehouse_lat'] = None
market_deliveries_df['warehouse_lon'] = None
market_deliveries_df['warehouse_id'] = None
market_deliveries_df['delivery_plan_dttm'] = market_orders_df.apply(
    lambda x: choose_random_time_in_interval(
        x['original_delivery_from_datetime'],
        x['original_delivery_to_datetime']
    ),
    axis=1
)
market_deliveries_df['order_id_source'] = market_orders_df['order_id'].astype(int)
market_deliveries_df['warehouse_ready_dttm'] = None
market_deliveries_df = market_deliveries_df[~pd.isnull(market_deliveries_df['delivery_lat'])]
market_deliveries_df = market_deliveries_df[~pd.isnull(market_deliveries_df['delivery_lon'])]

In [ ]:
market_deliveries_df = market_deliveries_df[market_deliveries_df['weight']<=10]

In [ ]:
market_deliveries_df.head()

In [ ]:
# # Pickup Points.
# pickup_points_df = pd.DataFrame()
# pickup_points_df['pickup_point_id'] = np.arange(len(lavka_places_df))
# pickup_points_df['lat'] = lavka_places_df['location_lat']
# pickup_points_df['lon'] = lavka_places_df['location_lon']
# pickup_points_df['take_start_tm'] = '06:00:00'
# pickup_points_df['take_end_tm'] = '07:00:00'
# pickup_points_df['address'] = lavka_places_df['address_full']

In [ ]:
daily_market_volume_moscow = 17200
daily_market_volume_kazan = 500
# deliveries_df = pd.concat(
#     [market_deliveries_df.sample(n=daily_market_volume_kazan, random_state=7)],
#     axis=0
# )

In [ ]:
real_pp_df = pd.read_excel('./import_data/pp_list.xlsx', skiprows=3)
real_kazan_pp_df = real_pp_df[(real_pp_df['Населённый пункт']=='г Казань')
& (real_pp_df['Тип пункта']=='ПВЗ')][['Широта', 'Долгота']].drop_duplicates(subset=('Широта', 'Долгота')).reset_index(
).rename(
    {'index': 'real_pp_id', 'Широта': 'lat', 'Долгота': 'lon'}, axis=1
)

### Find optimal number of points

In [ ]:
simulation_paramaters = {
    'Market_x0.5': {'frac_lavka_orders': 0., 'frac_market_orders': int(500/2)},
    'Market_x1': {'frac_lavka_orders': 0., 'frac_market_orders': int(500)},
    'Market_x2': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*2)},
    'Market_x3': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*3)},
    'Market_x4': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*4)},
    'Market_x5': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*5)},
    'Market_x6': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*6)},
    'Market_x7': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*7)},
    'Market_x8': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*8)},
    'Market_x9': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*9)},
    'Market_x10': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*10)},
#     'Market_x11': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*11)},
#     'Market_x12': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*12)},
#     'Market_x13': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*13)},
#     'Market_x14': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*14)},
#     'Market_x15': {'frac_lavka_orders': 0., 'frac_market_orders': int(500*15)},
}

In [ ]:
simulation_dict = {}
stats = {}

for key, value in simulation_paramaters.items():
    for sla in [10, 20, 30]:
        radius = sla * 132.1
        simulation_dict[(sla, key)] = {}

        # Initialize orders.
        deliveries_df = pd.concat(
            [market_deliveries_df.sample(n=value['frac_market_orders'], random_state=7)],
            axis=0
        )
        
        sim = common.Simulation(sla_seconds=sla*60, sh_cost=250, pp_work_time=14)
        simulation_dict[(key, sla)] = {}
        simulation_dict[(key, sla)]['simulation'] = sim
        sim.initialize_deliveries(deliveries_df)
        deliveries = sim.deliveries
        sim.deliveries = []
        sim.deliveries_dict = {}
        X = np.array(
            [
                [delivery.location_lat, delivery.location_lon]
                for delivery in deliveries
            ],
        )
        
        print('volume', deliveries_df.shape[0])

        # Choose number of clusters.
        num_clusters = math.ceil(deliveries_df.shape[0] / 120)
        print('parameters of iteration', key, sla, num_clusters)
    
        success_deliveries_frac = 0
        # Optimize pickup points.
        while success_deliveries_frac < 0.8:
            kclusterer = KMeansClusterer(num_clusters,
                                         distance=haversine.haversine,
                                         repeats=1,
                                         avoid_empty_clusters=True,)
            assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

            failed_deliveries_cnt = 0
            ac_points_in_radius = {}
            for ac, d in zip(assigned_clusters, deliveries):
                dist_ac = haversine.haversine(
                    kclusterer.means()[ac], (d.location_lat, d.location_lon)
                ) * 1000.
                if dist_ac > radius:
                    failed_deliveries_cnt += 1
            
            success_deliveries_frac = (len(deliveries_df) - failed_deliveries_cnt) / len(deliveries_df)
            if success_deliveries_frac < 0.9:
                num_clusters += 1
                print('increased num clusters to {}'.format(num_clusters))

        # Get pp df.
        pp_addresses = pd.DataFrame(
            {
                'pp_id': list(range(len(kclusterer.means()))),
                'lat': [c[0] for c in kclusterer.means()],
                'lon': [c[1] for c in kclusterer.means()],
            }
        )
        indexes = pairwise_distances(
            pp_addresses[['lat', 'lon']].values,
            real_kazan_pp_df[['lat', 'lon']].values,
            metric=haversine.haversine,
            force_all_finite=True
        ).argmin(axis=1)
        pp_addresses_real = real_kazan_pp_df.iloc[indexes]
        
        
        pickup_points_df = pd.DataFrame()
        pickup_points_df['pickup_point_id'] = 1000 + pp_addresses_real['real_pp_id']
        pickup_points_df['lat'] = pp_addresses_real['lat']
        pickup_points_df['lon'] = pp_addresses_real['lon']
        pickup_points_df['take_start_tm'] = '06:00:00'
        pickup_points_df['take_end_tm'] = '07:00:00'
        pickup_points_df['address'] = None
        
        # Run simulation.
        sim.initialize_simulation(deliveries_df, pickup_points_df)
        sim.assign_objects()
        sim.batch_deliveries_pp_to_user()
        sim.pay_couriers()
        stats[(key, sla, 'all')] = sim.calculate_metrics(sim.deliveries)
        print('\n')

In [ ]:
sum([c.time_travelled + 600*3 for c in sim.couriers]) / 3600

In [ ]:
pd.DataFrame(stats).T

In [ ]:
pd.DataFrame(stats).T.to_excel(
    './analytics/kazan_sla{sla}min_{pp_type}pp_{pp_cnt}pvz_new.xlsx'
    .format(sla='custom', pp_type='real', pp_cnt='custom')
)

### Draw orders on map

In [ ]:
sim = simulation_dict[('Market_x5', 10)]['simulation']

In [ ]:
YA_MAPS_TILES = 'http://vec02.maps.yandex.net/tiles?l=map&v=2.2.3&x={x}&y={y}&z={z}'

In [ ]:
f_map = folium.Map(
    location=(55.8, 49), zoom_start=10, tiles=YA_MAPS_TILES, max_zoom=20,
    attr='My Data Attribution', crs='EPSG3395'
)

In [ ]:
for d in sim.deliveries:
    c = 'red'
    if d.delivered_flg:
        c = 'blue'
    folium.CircleMarker(
        location=(d.location_lat, d.location_lon),
        radius=0.001,
        color=c,
        fill=True,
        weight=0.99,
    ).add_to(f_map)

In [ ]:
f_map.save('./kazan_market_x5.html')